In [20]:
import DataRetriever as dr
import pandas as pd

year_2 = pd.read_csv("C:/Users/madsc/OneDrive - Aalborg Universitet/P6/Data/raw_data/All-Subsystems-minute-Year2.csv",
                     parse_dates=["Timestamp"], infer_datetime_format=True, index_col=["Timestamp"])

In [21]:
year_2.index = pd.to_datetime(year_2.index, utc=True)

In [24]:
del year_2["DayOfWeek"]
year_2[year_2 > 0] = 1
year_2[year_2 <= 0] = 0

In [25]:
year_2 = year_2.resample(rule="H").mean()

In [26]:
year_2 = year_2.round()

In [ ]:
year_2